In [1]:
from mdagent import MDAgent

In [2]:
prompt5 = "Simulate 1VII for 1 ns at a temperature of 300 K. Then, tell me if the secondary structure changed from the beginning of the simulation to the end of the simulation."
llm_var = "claude-3-opus-20240229"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools, use_memory=False)
print(llm_var)

claude-3-opus-20240229


In [3]:
agent.run(prompt5)

Thought: To simulate 1VII for 1 ns at 300 K and analyze the secondary structure changes, I will first download the PDB file, then set up and run a 1 ns simulation at 300 K. After the simulation, I will use the ComputeDSSP tool to analyze the secondary structure at the first and last frames, and compare them to determine if there were any changes.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1VII"
}
```

PDB file found with this ID: 1VII
Here are the next steps to continue the simulation and analysis:

Thought: Now that I have the PDB file, I will set up and run a 1 ns simulation at 300 K using the SetUpandRunFunction tool. I'll specify the PDB file ID, forcefield files, and simulation parameters like temperature, number of steps, and recording interval.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1VII_185208",
    "forcefield_files": ["amber14/protein.ff14SB.xml", "amber14/tip3p.xml"],
    "system_params": {
      "nonbonde

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [6]:
registry = agent.path_registry
all_paths = registry.list_path_names_and_descriptions()
print (all_paths)
assert "PDBFile ID: 1VII" in all_paths
assert all([file_id in all_paths for file_id in ["rec0_", "rec1_", "rec2_", "dssp values", "sim0_", "top_sim0"]])

Files found in registry: 1VII_185208: PDB file downloaded from RSCB, PDBFile ID: 1VII_185208, top_sim0_185311: Initial positions for simulation sim0_185311, sim0_185311: Basic Simulation of Protein 1VII_185208, rec0_185312: Simulation trajectory for protein 1VII_185208 and simulation sim0_185311, rec1_185312: Simulation state log for protein 1VII_185208 and simulation sim0_185311, rec2_185312: Simulation pdb frames for protein 1VII_185208 and simulation sim0_185311, rec0_200812: dssp values for trajectory with id: rec0_185312, rec0_200819: dssp values for trajectory with id: rec0_185312


In [8]:
import mdtraj as md

traj_path = registry.get_mapped_path("rec0_185312")
top_path = registry.get_mapped_path("top_sim0_185311")
traj = md.load(traj_path, top=top_path)

# Compute the secondary structure of the trajectory
dssp_final = md.compute_dssp(traj[-1], simplified=True)
dssp_initial = md.compute_dssp(traj[0], simplified=True)

print("Number of initial sheets: ",len([i for i in dssp_initial[0] if i == 'E']))
print("Number of initial helices: ",len([i for i in dssp_initial[0] if i == 'H']))
print("Number of initial coils: ",len([i for i in dssp_initial[0] if i == 'C']))

print("Number of final sheets: ",len([i for i in dssp_final[0] if i == 'E']))
print("Number of final helices: ",len([i for i in dssp_final[0] if i == 'H']))
print("Number of final coils: ",len([i for i in dssp_final[0] if i == 'C']))

Number of initial sheets:  0
Number of initial helices:  21
Number of initial coils:  15
Number of final sheets:  0
Number of final helices:  22
Number of final coils:  14
